In [1]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.autograd import Variable
import torch.nn as nn
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

epochs = 10
hidden_size = 20
intermediate_size = 128
log_interval = 10
device = torch.device("cpu")


kwargs = {'num_workers': 1, 'pin_memory': True} 
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('../data', train=True, download=True,
                     transform=transforms.ToTensor()),
    batch_size=512, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('../data', train=False, transform=transforms.ToTensor()),
    batch_size=512, shuffle=False, **kwargs)


class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        # Encoder
        self.conv1 = nn.Conv2d(3, 3, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(3, 32, kernel_size=2, stride=2, padding=0)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(16 * 16 * 32, intermediate_size)

        # Latent space
        self.fc21 = nn.Linear(intermediate_size, hidden_size)
        self.fc22 = nn.Linear(intermediate_size, hidden_size)

        # Decoder
        self.fc3 = nn.Linear(hidden_size, intermediate_size)
        self.fc4 = nn.Linear(intermediate_size, 8192)
        self.deconv1 = nn.ConvTranspose2d(32, 32, kernel_size=3, stride=1, padding=1)
        self.deconv2 = nn.ConvTranspose2d(32, 32, kernel_size=3, stride=1, padding=1)
        self.deconv3 = nn.ConvTranspose2d(32, 32, kernel_size=2, stride=2, padding=0)
        self.conv5 = nn.Conv2d(32, 3, kernel_size=3, stride=1, padding=1)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def encode(self, x):
        out = self.relu(self.conv1(x))
        out = self.relu(self.conv2(out))
        out = self.relu(self.conv3(out))
        out = self.relu(self.conv4(out))
        out = out.view(out.size(0), -1)
        h1 = self.relu(self.fc1(out))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = Variable(std.data.new(std.size()).normal_())
            return eps.mul(std).add_(mu)
        else:
            return mu

    def decode(self, z):
        h3 = self.relu(self.fc3(z))
        out = self.relu(self.fc4(h3))
        # import pdb; pdb.set_trace()
        out = out.view(out.size(0), 32, 16, 16)
        out = self.relu(self.deconv1(out))
        out = self.relu(self.deconv2(out))
        out = self.relu(self.deconv3(out))
        out = self.sigmoid(self.conv5(out))
        return out

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


model = VAE()

optimizer = optim.RMSprop(model.parameters(), lr=1e-3)


# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x.view(-1, 32 * 32 * 3),
                                 x.view(-1, 32 * 32 * 3), size_average=False)

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD


def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = Variable(data)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.data[0]
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.data[0] / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


def test(epoch):
    model.eval()
    test_loss = 0
    for i, (data, _) in enumerate(test_loader):

        data = Variable(data, volatile=True)
        recon_batch, mu, logvar = model(data)
        test_loss += loss_function(recon_batch, data, mu, logvar).data[0]
        if epoch == epochs and i == 0:
            n = min(data.size(0), 8)
            comparison = torch.cat([data[:n],
                                   recon_batch[:n]])
            save_image(comparison.data.cpu(),
                       'snapshots/conv_vae/reconstruction_' + str(epoch) +
                       '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

Files already downloaded and verified


In [1]:
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)
    if epoch == epochs:
        sample = Variable(torch.randn(64, hidden_size))
        sample = model.decode(sample).cpu()
        save_image(sample.data.view(64, 3, 32, 32),
                   './sample_' + str(epoch) + '.png')

Files already downloaded and verified


/Users/gexinyue/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
/Users/gexinyue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:141: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/Users/gexinyue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:147: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Train Epoch: 1 [0/50000 (0%)]	Loss: 2124.020020
Train Epoch: 1 [5120/50000 (10%)]	Loss: 2119.039795
Train Epoch: 1 [10240/50000 (20%)]	Loss: 2109.692871
Train Epoch: 1 [15360/50000 (31%)]	Loss: 2135.869629
Train Epoch: 1 [20480/50000 (41%)]	Loss: 2137.454590
Train Epoch: 1 [25600/50000 (51%)]	Loss: 2104.304688
Train Epoch: 1 [30720/50000 (61%)]	Loss: 2132.151611
Train Epoch: 1 [35840/50000 (71%)]	Loss: 2124.375732
Train Epoch: 1 [40960/50000 (82%)]	Loss: 2117.572998
Train Epoch: 1 [46080/50000 (92%)]	Loss: 2104.397217
====> Epoch: 1 Average loss: 2125.4067


/Users/gexinyue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:158: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/Users/gexinyue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:160: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


====> Test set loss: 2118.7556
Train Epoch: 2 [0/50000 (0%)]	Loss: 2077.984131
Train Epoch: 2 [5120/50000 (10%)]	Loss: 2014.935669
Train Epoch: 2 [10240/50000 (20%)]	Loss: 2019.218384
Train Epoch: 2 [15360/50000 (31%)]	Loss: 2006.380981
Train Epoch: 2 [20480/50000 (41%)]	Loss: 2040.215332
Train Epoch: 2 [25600/50000 (51%)]	Loss: 2014.866089
Train Epoch: 2 [30720/50000 (61%)]	Loss: 2024.642090
Train Epoch: 2 [35840/50000 (71%)]	Loss: 2000.441406
Train Epoch: 2 [40960/50000 (82%)]	Loss: 1949.568970
Train Epoch: 2 [46080/50000 (92%)]	Loss: 1957.253540
====> Epoch: 2 Average loss: 2003.9983
====> Test set loss: 1961.0876
Train Epoch: 3 [0/50000 (0%)]	Loss: 1960.758179
Train Epoch: 3 [5120/50000 (10%)]	Loss: 1955.104858
Train Epoch: 3 [10240/50000 (20%)]	Loss: 1945.156006
Train Epoch: 3 [15360/50000 (31%)]	Loss: 1943.620728
Train Epoch: 3 [20480/50000 (41%)]	Loss: 1988.982178
Train Epoch: 3 [25600/50000 (51%)]	Loss: 1958.351318
Train Epoch: 3 [30720/50000 (61%)]	Loss: 1945.795898
Train Epoc

FileNotFoundError: [Errno 2] No such file or directory: 'snapshots/conv_vae/reconstruction_10.png'

In [3]:
torch.save(model, './cifar_model')

/Users/gexinyue/anaconda3/lib/python3.6/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type VAE. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [4]:
model = torch.load('./cifar_model')

In [7]:
sample = Variable(torch.randn(64, hidden_size))
sample = model.decode(sample).cpu()
save_image(sample.data.view(64, 3, 32, 32),'Hello.png')